### 내 사전 감성점수 파악(시드 단어목록과 부트스트래핑)

In [ ]:
import numpy as np
from gensim.models import FastText
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import os
import random
from scipy.spatial.distance import cosine

# 모델 로드
model_path = "업데이트.bin"  # 실제 경로로 수정
try:
    model = FastText.load(model_path)
except:
    print(f"모델 로드 실패. '{model_path}' 경로를 확인하세요.")
    exit()

# 시드 단어 정의
hawkish_words = [
    "강경", "공격적", "제재", "압박", "보복", "군사", "대응", "대립", "분쟁", "전쟁", "위험", "자극",
    "불안", "위기", "공격", "반격", "저항", "독재", "억제", "억압", "단호", "무력", "충돌", "전술",
    "철강", "경제적 압박", "무역전쟁", "대립각", "강경책", "복수", "강력한", "신속한", "도발", "세력", "공격적인","높","인상","매파","성장","투기",
    "상승","인플레이션","물가 상승", "긴축", "흑자", "채권", "우려","압력", "금리 상승", "금리 인상", "긴축 변동성 감소", "낙관 부동산 가격 상승", "부동산", "크", "높"
]

dovish_words = [
    "평화", "협력", "완화", "타협", "외교", "대화", "조정", "중재", "합의", "화해", "안정", "협상",
    "협약", "교류", "관계", "회담", "평화적", "소통", "융화", "유화", "국제적", "상호이해", "신뢰",
    "공동체", "존중", "연대", "공존", "구호", "공동선", "평화주의", "유연한", "지속적", "소극적", "융통성",
    "친선", "구속력", "구속적", "비둘기파" , "낮", "축소", "인하", "둔화", "약화", "하락", "회복", "못하", "하회 물가 하락", "위험", "물가 하락", "침체", "하방", "완화", "변동성 확대", "부진", "요금 인하", "부동산 가격 하락", "하향", "적자","채권", "가격 상승", "최악", "물가상승"
]

# 부트스트래핑 설정
num_bootstrap_iterations = 50  # 반복 횟수
num_subsets = 10  # 하위 집합 개수
max_words_per_category = 50000  # 최대 단어 수 제한

# 감성 단어 벡터 계산 함수
def get_average_vector(words, model):
    vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# 감성 단어를 하위 집합으로 나누기
def split_into_subsets(words, num_subsets):
    random.shuffle(words)
    return [words[i::num_subsets] for i in range(num_subsets)]

hawkish_subsets = split_into_subsets(hawkish_words, num_subsets)
dovish_subsets = split_into_subsets(dovish_words, num_subsets)

# 단어 극성 점수 계산 함수
def compute_score(word, hawkish_vector, dovish_vector):
    if word not in model.wv:
        return 0.0
    word_vector = model.wv[word]
    hawkish_sim = 1 - cosine(word_vector, hawkish_vector)
    dovish_sim = 1 - cosine(word_vector, dovish_vector)
    score = hawkish_sim / dovish_sim if dovish_sim != 0 else 0.0
    return score

# 부트스트래핑을 통한 감성 전파
def bootstrap_polarity_lexicon(model, num_iterations, num_subsets):
    global hawkish_words, dovish_words
    
    for iteration in range(num_iterations):
        print(f"부트스트래핑 반복 {iteration + 1} / {num_iterations}")
        for i in range(num_subsets):
            hawkish_vector = get_average_vector(hawkish_subsets[i], model)
            dovish_vector = get_average_vector(dovish_subsets[i], model)
            words = model.wv.index_to_key
            
            with ThreadPoolExecutor() as executor:
                scores = list(tqdm(executor.map(lambda w: compute_score(w, hawkish_vector, dovish_vector), words),
                                   total=len(words), desc=f"Subset {i + 1}/{num_subsets}"))
            
            new_hawkish, new_dovish = [], []
            for word, score in zip(words, scores):
                if 1.0 < score < 1.1:
                    continue  # 중립 영역 제외
                if score > 1.1:
                    new_hawkish.append(word)
                elif score < 0.9:
                    new_dovish.append(word)
            
            hawkish_words.extend(new_hawkish)
            dovish_words.extend(new_dovish)
            
            # 중복 제거 및 개수 제한
            hawkish_words = list(set(hawkish_words))[:max_words_per_category]
            dovish_words = list(set(dovish_words))[:max_words_per_category]
    
    return hawkish_words, dovish_words

# 실행
hawkish_lexicon, dovish_lexicon = bootstrap_polarity_lexicon(model, num_bootstrap_iterations, num_subsets)

# 결과 저장
output_dir = "polarity_bootstrap_limit"
os.makedirs(output_dir, exist_ok=True)
pd.DataFrame({"Hawkish": hawkish_lexicon}).to_csv(os.path.join(output_dir, "hawkish_words_limit.csv"), encoding='utf-8-sig', index=False)
pd.DataFrame({"Dovish": dovish_lexicon}).to_csv(os.path.join(output_dir, "dovish_words_limit.csv"), encoding='utf-8-sig', index=False)

print("부트스트래핑 완료.")
